In [52]:
import requests
import json
import pandas as pd
from datetime import date, datetime, timedelta
from itertools import product, combinations
import time

client authentication <u>do not make this public</u>

this is lazy security only becuase were using a free, capped api, production will need to be different

In [2]:
from amadeus import Client, ResponseError

amadeus = Client(
    client_id='eXya3OTX00wsuNXBGgp9KGPbLYwSOOGI',
    client_secret='ZtWWaZ3v56NtZAAp'
)

this is the search prep to get all the combos for the date range and airports

In [40]:
start_date = date(2025, 3, 22)
end_date = date(2025, 3, 31)

date_list = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end_date - start_date).days + 1)]

cities = ['ORD', 'MDW', 'LAX', 'SNA', 'LGA', 'EWR', 'JFK']

city_pairs_list = [list(pair) for pair in combinations(cities, 2)]

date_city_pairs_combo = [[date] + city_pair for date, city_pair in product(date_list, city_pairs_list)]


had to add a `time.sleep(1)` so that we wouldn't over load the api. i think this is a function of it being test and not production bc i think we'd get a higher rate at production

In [ ]:
index_no = 0

for i in date_city_pairs_combo:
    date = i[0]
    orig = i[1]
    dest = i[2]

    time.sleep(1)

    response = amadeus.shopping.flight_offers_search.get(
        originLocationCode=orig,
        destinationLocationCode=dest,
        departureDate=date,
        adults=1,
        travelClass='ECONOMY',
        nonStop='true',
        currencyCode='USD',
        max=100).result

    flight_data = []
    for i in range(len(response['data'])):
        data = {
                "maual_index": index_no,
                "dep_code": response['data'][i]['itineraries'][0]['segments'][0]['departure']['iataCode'],
                "dep_datetime": response['data'][i]['itineraries'][0]['segments'][0]['departure']['at'],
                "arv_code": response['data'][i]['itineraries'][0]['segments'][0]['arrival']['iataCode'],
                "arv_datetime": response['data'][i]['itineraries'][0]['segments'][0]['arrival']['at'],
                "airline_code": response['data'][i]['itineraries'][0]['segments'][0]['carrierCode'],
                "flight_no": response['data'][i]['itineraries'][0]['segments'][0]['number'],
                "price_total": response['data'][i]['price']['grandTotal'],
                "seats_remaining": response['data'][i]['numberOfBookableSeats'],
                "date_api_request": datetime.today().strftime('%Y-%m-%d')
            }
        if data['airline_code'] == '6X':
            continue
        else:
            flight_data.append(data)
            index_no += 1

    df = pd.DataFrame(flight_data)
    df.to_csv('flight_data_iter_2.csv', index=False, mode='a', header=False)
    print(f"wrote {date} -- {orig} -- {dest}")